In [2]:
import os
import scipy.io
import numpy as np
import pywt
import wfdb
import matplotlib.pyplot as plt
import scipy.signal as sg
import cv2
from pathlib import Path
from glob import glob
from concurrent.futures import ProcessPoolExecutor
import joblib

In [3]:
ROOT_PATH = "E:/pv/WORKING/ECG_main_folder/ECG_Classification/data/raw/mit-bih-arrhythmia-database-1.0.0"
OUTPUT_PATH = "./processed_images"
sampling_rate = 360
wavelet = "mexh"  # mexh, morl, gaus8, gaus4
scales = pywt.central_frequency(wavelet) * sampling_rate / np.arange(1, 101, 1)

# Danh sách tất cả file record (.dat)
record_files = sorted(glob(os.path.join(ROOT_PATH, '*.dat')))

cpus = 22 if joblib.cpu_count() > 22 else joblib.cpu_count() - 1  # for multi-process

In [4]:
def read_ecg_record(record):
    record_path = Path(ROOT_PATH)/ str(record) # Combine PATH and record using pathlib
    signal = wfdb.rdrecord(record_path.as_posix(), channels=[0]).p_signal[:, 0]
    annotation = wfdb.rdann(record_path.as_posix(), extension="atr")
    r_peaks, labels = annotation.sample, np.array(annotation.symbol)
    return signal,r_peaks,labels 

In [5]:
def median_filter(signal):
    baseline = sg.medfilt(sg.medfilt(signal, int(0.2 * sampling_rate) - 1), int(0.6 * sampling_rate) - 1)
    filtered_signal = signal - baseline 
    return filtered_signal

In [6]:
def remove_invalid_labels(r_peaks, labels):
    # Các nhãn không phải nhịp tim
    invalid_labels = ['|', '~', '!', '+', '[', ']', '"', 'x']
    indices = [i for i, label in enumerate(labels) if label not in invalid_labels]
        #enumerate(labels) sẽ tạo ra một danh sách các cặp (index, label)
        #labels = ['N', '|', 'V', '~'] → enumerate(labels) sinh ra (0, 'N'), (1, '|'), (2, 'V'), (3, '~')
        #Vòng lặp for i, label in enumerate(labels)
        #Điều kiện if label not in invalid_labels
        #  ==> Danh sách kết quả indices Chứa danh sách các chỉ số i của label hợp lệ
    r_peaks, labels = r_peaks[indices], labels[indices]
    return r_peaks, labels

In [7]:
def align_r_peaks(r_peaks, filtered_signal,tol=0.05):
    newR = []
        #danh sách rỗng newR để lưu vị trí đỉnh R đã được căn chỉnh
    for r_peak in r_peaks: #Lặp qua từng điểm đỉnh R (r_peak) đã phát hiện trước đó.
        r_left = np.maximum(r_peak - int(tol * sampling_rate), 0)
            #Xác định giới hạn trái: Lùi lại một khoảng tol * sampling_rate từ vị trí r_peak để tạo một cửa sổ tìm kiếm.
            #np.maximum(..., 0) để đảm bảo không bị âm (tránh lỗi khi r_peak ở đầu tín hiệu).
        r_right = np.minimum(r_peak + int(tol * sampling_rate), len(filtered_signal))
            #Xác định giới hạn phải
        newR.append(r_left + np.argmax(filtered_signal[r_left:r_right]))
    r_peaks = np.array(newR, dtype="int") # ép kiểu newR về int 

    #normalize signal 
    normalized_signal = filtered_signal / np.mean(filtered_signal[r_peaks])
    return r_peaks, normalized_signal

In [8]:
# Nhóm các label thành 5 class
def AAMI_categories(labels): 
    AAMI = {
        "N": 0, "L": 0, "R": 0, "e": 0, "j": 0,  # N
        "A": 1, "a": 1, "S": 1, "J": 1,  # SVEB
        "V": 2, "E": 2,  # VEB
        "F": 3,  # F
        "/": 4, "f": 4, "Q": 4  # Q
    }
    categories = [AAMI[label] for label in labels]
        #Nếu label = "N", thì AAMI["N"] sẽ trả về 0
        #[AAMI[label] for label in labels] là list comprehension trong Python, nó tạo ra một list mới. 
        #Kết quả là một danh sách các giá trị mã phân loại tương ứng với từng nhãn trong labels.
    return categories

In [40]:
# hàm này returns:
    # beats (list of np.ndarray): Danh sách đoạn nhịp ECG đã cắt
    # beat_labels (list): Nhãn tương ứng với từng nhịp
def segmentation(normalize_signal, r_peaks, categories):
    before, after = 90, 110 ## Lấy đoạn tín hiệu 200ms quanh R-peak
    beats, beat_labels = [], []
    for r, category in zip(r_peaks, categories):
        start = r - before
        end = r + after
        if category!=4:
            if start >= 0 and end < len(normalize_signal): # nếu không đủ dữ liệu (gần biên), bỏ qua
                beat = normalize_signal[start:end]
                beats.append(beat)
                beat_labels.append(category)
        
    return beats, beat_labels


In [ ]:
from tqdm import tqdm #tqdm dùng để hiện progress bar khi xử lý nhiều ảnh
from PIL import Image
from concurrent.futures import ProcessPoolExecutor
from functools import partial

def CWT(record_name, beats, beat_labels, image_size = 100):

    os.makedirs(OUTPUT_PATH, exist_ok=True) #Tạo thư mục gốc để lưu ảnh (nếu chưa tồn tại). Dùng exist_ok=True để tránh lỗi nếu thư mục đã có
    
    for i, (beat, beat_label) in enumerate(tqdm(zip(beats, beat_labels), total=len(beats))):
            #zip(beats, beat_labels)	Gộp hai list beats và beat_labels lại thành các cặp (beat, label)
            #tqdm(...)	Hiển thị progress bar

        # Chuyển CWT
        # 1. Tính CWT → scalogram
        coef, _ = pywt.cwt(beat, scales, wavelet)
        scalogram = np.abs(coef)
        # 2. Chuẩn hóa về [0, 255] để lưu ảnh
        scalogram -= np.min(scalogram)
        scalogram /= np.max(scalogram) + 1e-6 #+1e-6: tránh chia cho 0
        scalogram *= 255
        scalogram = scalogram.astype(np.uint8) #Đổi sang uint8: đúng định dạng ảnh grayscale.
        # 3. Resize ảnh
        resized = cv2.resize(scalogram, (image_size, image_size), interpolation=cv2.INTER_CUBIC) #dùng nội suy INTER_CUBIC để giữ chất lượng cao
        # 4. Tạo thư mục theo label
        label_dir = os.path.join(OUTPUT_PATH, str(beat_label))
        os.makedirs(label_dir, exist_ok=True)
        # 5. Lưu ảnh dưới dạng .png (grayscale)
        save_path = os.path.join(label_dir, f'{record_name}_{beat_label}_{i}.png')
        cv2.imwrite(save_path, resized)  # ảnh grayscale


### thử trên 1 record 100

In [11]:
signal100,r_peaks100,labels100 = read_ecg_record(100)       
signal100

array([-0.145, -0.145, -0.145, ..., -0.675, -0.765, -1.28 ],
      shape=(650000,))

In [12]:
filtered100 = median_filter(signal100)
# print(filtered100)
# plt.plot(filtered100[:1000])

In [13]:
r_peaks100, labels100 = remove_invalid_labels(r_peaks100,labels100)
r_peaks100
labels100

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], shape=(2273,), dtype='<U1')

In [14]:
r_peaks100, normalize_signal100 = align_r_peaks(r_peaks100,filtered100)
#plt.plot(normalize_signal100[:1000])

In [15]:
categories100 = AAMI_categories(labels100)
#categories100

In [16]:
# sau khi đã filter 
print(normalize_signal100 , len(normalize_signal100))
print(r_peaks100 , len(r_peaks100))
print(len(categories100))

[-0.01850215 -0.00740086 -0.00740086 ... -0.499558   -0.56616573
 -0.94730998] 650000
[    77    370    663 ... 649485 649734 649991] 2273
2273


In [ ]:
beats100, beat_labels100 = segmentation(normalize_signal100,r_peaks100,categories100)
#print(len(beats100))
#print(len(beat_labels100))

2271


In [22]:
CWT(beats100, beat_labels100)

100%|██████████| 2271/2271 [00:06<00:00, 338.17it/s]


### Hàm MAIN

In [112]:

# for record_file in tqdm(record_files):
#         record_name = os.path.splitext(os.path.basename(record_file))[0]
#         print(f'\n Processing record: {record_name}')

#         # 1. Load dữ liệu
#         signal,r_peaks,labels = read_ecg_record(record_name) 
#         # 2. Lọc tín hiệu
#         filtered_signal = median_filter(signal)
#         # 3. lọc nhãn invalid
#         r_peaks, labels = remove_invalid_labels(r_peaks,labels)
#         # 4. căn chỉnh R, chuẩn hóa
#         r_peaks, normalize_signal = align_r_peaks(r_peaks,filtered_signal)
#         # 5. đổi sang dạng AAMI
#         categories = AAMI_categories(labels)
#         # 6. segmentation
#         beats, beat_labels = segmentation(normalize_signal,r_peaks,categories)
#         # 7. Chuyển thành scalogram và lưu
#         CWT(beats, beat_labels)
# print('\n✅ Hoàn thành xử lý toàn bộ records!')

In [ ]:
all_records = [
        '100', '101', '103', '105', '106', '108', '109', '111', '112', '113',
        '114', '115', '116', '117', '118', '119', '121', '122', '123', '124',
        '200', '201', '202', '203', '205', '207', '208', '209', '210', '212',
        '213', '214', '215', '219', '220', '221', '222', '223', '228', '230',
        '231', '232', '233', '234'
    ]

for record_name in all_records:

    print(f'\n Processing record: {record_name}')

    # 1. Load dữ liệu
    signal,r_peaks,labels = read_ecg_record(record_name) 
    # 2. Lọc tín hiệu
    filtered_signal = median_filter(signal)
    # 3. lọc nhãn invalid
    r_peaks, labels = remove_invalid_labels(r_peaks,labels)
    # 4. căn chỉnh R, chuẩn hóa
    r_peaks, normalize_signal = align_r_peaks(r_peaks,filtered_signal)
    # 5. đổi sang dạng AAMI
    categories = AAMI_categories(labels)
    # 6. segmentation
    beats, beat_labels = segmentation(normalize_signal,r_peaks,categories)
    # 7. Chuyển thành scalogram và lưu
    CWT(record_name,beats, beat_labels)


 Processing record: 100


100%|██████████| 2271/2271 [00:17<00:00, 131.63it/s]



 Processing record: 101


100%|██████████| 1862/1862 [00:13<00:00, 138.30it/s]



 Processing record: 103


...

100%|██████████| 1780/1780 [00:12<00:00, 140.52it/s]



 Processing record: 233


100%|██████████| 3077/3077 [00:21<00:00, 141.63it/s]



 Processing record: 234


100%|██████████| 2753/2753 [00:20<00:00, 134.24it/s]
